In [9]:
import numpy as np
from numpy import pi
import time
import kwant
import kwant.continuum
import tinyarray
import scipy.sparse.linalg as sla
from LK_utils import consts, Jmatrices, H_LKBP, get_material_parameters
from double_dot import Gaussian_symmetric_double_dot

# define box of computation (lengths in nm)

L = 240
W = 120
H = 20

# Parameter for definition of computation lattice

ax, ay, az = 4, 4, 1

lat_params = {"L": L, "W": W, "H": H, "ax": ax, "ay": ay, "az": az}
print('Lattice parameters (lengths in nm): {}'.format(lat_params))

nx, ny, nz = int(L/ax - 1), int(W/ay - 1), int(H/az - 1)
print('Number of points on the lattice: {0}*{1}*{2}={3}'.format(nx, ny, nz, nx*ny*nz))

print(H_LKBP)

Lattice parameters (lengths in nm): {'L': 240, 'W': 120, 'H': 20, 'ax': 4, 'ay': 4, 'az': 1}
Number of points on the lattice: 59*29*19=32509

    mu * ((ga1 + 5*ga2/2) * (k_x**2 + k_y**2 + k_z**2 + 2 * k_x * Ax(y,z) + 2 * k_y * Ay(x,z)) * id4 
    - 2 * ga2 * (k_x**2 * Jx2 + k_y**2 * Jy2 + k_z**2 * Jz2 + 2 * k_x * Ax(y,z) * Jx2 + 2 * k_y * Ay(x,z) * Jy2) 
    - ga3 * (kyz * Jyz + kzx * Jzx + kxy * Jxy 
    + (k_z * Ay(x,z) + Ay(x,z) * k_z) * Jyz + (k_z * Ax(y,z) + Ax(y,z) * k_z) * Jzx 
    + (k_x * Ay(x,z) + Ax(y,z) * k_y + k_y * Ax(y,z) + Ay(x,z) * k_x) * Jxy)) 
    + 2 * muB * (ka * (Jx*Bx + Jy*By + Jz*Bz) + qu * (Jx3*Bx + Jy3*By + Jz3*Bz)) 
    + V(x, y, z) * id4
    + bv * (epsxx * Jx2 + epsyy * Jy2 + epszz * Jz2)
    


In [25]:
def make_model_LKBP(H_LK, mat_params, lat_params, eps_para, eps_perp, B):
    syst = kwant.Builder()
    subs = consts
    subs.update(mat_params)
    subs.update(lat_params)
    subs.update(Jmatrices)
    subs.update({
        "Bx": B[0], "By": B[1], "Bz": B[2],
        "kyz": "k_y * k_z + k_z * k_y",
        "kzx": "k_z * k_x + k_x * k_z",
        "kxy": "k_x * k_y + k_y * k_x",
        "epsxx": eps_para, "epsyy": eps_para, "epszz":eps_perp
        })

    model = kwant.continuum.discretize(H_LK, locals=subs, grid=kwant.lattice.general([(lat_params['ax'], 0, 0), (0, lat_params['ay'], 0), (0, 0, lat_params['az'])], norbs=4))
    def box_3D(site):
        (x, y, z) = site.pos
        return 0 <= x < lat_params['L'] - lat_params['ax'] and 0 <= y < lat_params['W'] - lat_params['ay'] and 0 <= z < lat_params['H'] - lat_params['az']

    syst.fill(model, box_3D, (0, 0, 0))
    syst = syst.finalized()
    return syst


def eigs_gate_defined_potential(syst, d_QW, B, V, N=8):
    def potential_vector_x(y, z):
        return (2*pi/consts['phi0']) * (z*B[1] - y*B[2]/2)
    
    def potential_vector_y(x, z):
        return (2*pi/consts['phi0']) * (-z*B[0] + x*B[2]/2)
    
    def potential(x, y, z):
        def g(x,y,z):
            return (0.5/pi) * np.arctan2(x * y, (z * np.sqrt(x**2 + y**2 + z**2)))
        
        Left = x0 - 0.5*Lg # Left and right x-coordinates of the dot
        Right = x0 + 0.5*Lg  
        Bot = y0 - 0.5*Lg # Bottom and top y-coordinates of the dot
        Top = y0 + 0.5*Lg
        return V * (g(x - Left, y - Bot, z + d_QW) + g(x - Left, Top - y, z + d_QW) + g(Right - x, y - Bot, z + d_QW) + g(Right - x, Top - y, z + d_QW))
        
    Ham = syst.hamiltonian_submatrix(params=dict(V=potential, Ax=potential_vector_x, Ay=potential_vector_y), sparse=True)
    evals, evecs = sla.eigsh(Ham, k=N, which='SA')
    idx = evals.argsort()
    return [evals[idx], evecs[:,idx]]


def eigs_double_dot_potential(syst, B, d, s, U0, eEz, N=8):
    def potential_vector_x(y, z):
        return (2*pi/consts['phi0']) * (z*B[1] - y*B[2])
    
    def potential_vector_y(x, z):
        return (2*pi/consts['phi0']) * (-z*B[0])
    
    def double_dot_potential(x, y, z):
        return Gaussian_symmetric_double_dot(x, y, d, U0, s) + eEz*z

    Ham = syst.hamiltonian_submatrix(params=dict(V=double_dot_potential, Ax=potential_vector_x, Ay=potential_vector_y), sparse=True)
    evals, evecs = sla.eigsh(Ham, k=N, which='SA')
    idx = evals.argsort()
    return [evals[idx], evecs[:,idx]]


def D_dip(eigs, lat_params):
    evals = eigs[0]
    Ds = evals[1] - evals[0]
    Dc = 0.5 * (evals[2] + evals[3] - evals[0] - evals[1])
    print('Dc = {0} meV, Ds = {1} meV'.format(Dc, Ds))
    
    norbs = 4
    x = np.arange(0, L - ax, ax)
    y = np.arange(0, W - ay, ay)
    z = np.arange(0, H - az, az)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    grid_shape = np.shape(X)
    
    # Get the wavefunctions for the first two levels
    psi0 = eigs[1][:, 0].reshape((int(L/ax - 1) * int(W/ay - 1) * int(H/az - 1), norbs))
    psi1 = eigs[1][:, 1].reshape((int(L/ax - 1) * int(W/ay - 1) * int(H/az - 1), norbs))
    
    # Compute matrix element for each spinor component
    integral_x01 = integral_y01 = integral_z01 = integral_xl = integral_yl = integral_zl = 0.0 + 0.0j 
    norm0 = norm1 = 0.0
    for i in range(4):
        psi0_i = psi0[:, i].reshape(grid_shape)
        psi1_i = psi1[:, i].reshape(grid_shape)
        norm0 += np.sum(np.conj(psi0_i)*psi0_i)
        norm1 += np.sum(np.conj(psi1_i)*psi1_i)
        integral_x01 += np.sum(np.conj(psi0_i) * X * psi1_i)
        integral_y01 += np.sum(np.conj(psi0_i) * Y * psi1_i) 
        integral_z01 += np.sum(np.conj(psi0_i) * Z * psi1_i)
        integral_xl += np.sum(np.conj(psi1_i) * X * psi1_i - np.conj(psi0_i) * X * psi0_i) 
        integral_yl += np.sum(np.conj(psi1_i) * Y * psi1_i - np.conj(psi0_i) * Y * psi0_i) 
        integral_zl += np.sum(np.conj(psi1_i) * Z * psi1_i - np.conj(psi0_i) * Z * psi0_i)
    
    evals = eigs[0]
    Dc = 0.5 * (evals[2] + evals[3] - evals[0] - evals[1])
    Ds = evals[1] - evals[0]
    
    norbs = 4
    x = np.arange(0, L - ax, ax)
    y = np.arange(0, W - ay, ay)
    z = np.arange(0, H - az, az)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    grid_shape = np.shape(X)
    
    # Get the wavefunctions for the first two levels
    psi0 = eigs[1][:, 0].reshape((int(L/ax - 1) * int(W/ay - 1) * int(H/az - 1), norbs))
    psi1 = eigs[1][:, 1].reshape((int(L/ax - 1) * int(W/ay - 1) * int(H/az - 1), norbs))
    
    # Compute matrix element for each spinor component
    integral_x01 = integral_y01 = integral_z01 = integral_xl = integral_yl = integral_zl = 0.0 + 0.0j 
    N0 = N1 = 0.0 + 0.0j
    
    for i in range(4):
        psi0_i = psi0[:, i].reshape(grid_shape)
        psi1_i = psi1[:, i].reshape(grid_shape)
        N0 += np.sum(np.conj(psi0_i)*psi0_i)
        N1 += np.sum(np.conj(psi1_i)*psi1_i)
        integral_x01 += np.sum(np.conj(psi0_i) * X * psi1_i)
        integral_y01 += np.sum(np.conj(psi0_i) * Y * psi1_i) 
        integral_z01 += np.sum(np.conj(psi0_i) * Z * psi1_i)
        integral_xl += np.sum(np.conj(psi1_i) * X * psi1_i - np.conj(psi0_i) * X * psi0_i) 
        integral_yl += np.sum(np.conj(psi1_i) * Y * psi1_i - np.conj(psi0_i) * Y * psi0_i) 
        integral_zl += np.sum(np.conj(psi1_i) * Z * psi1_i - np.conj(psi0_i) * Z * psi0_i)

    if max(abs(N0), abs(N1))<1e-3:
        print('Normalization OK')
    
    x01 = np.abs(integral_x01)
    y01 = np.abs(integral_y01)
    z01 = np.abs(integral_z01)
    xl = np.abs(integral_xl)
    yl = np.abs(integral_yl)
    zl = np.abs(integral_zl)
    return [Dc, Ds, x01, y01, z01, xl, yl, zl]


In [27]:
mat = 'Ge'
mat_params = get_material_parameters(mat)
print('Material parameters: {}'.format(mat_params))

B=[0.25, 0.25, 0.05]

d, s, U0, eEz = 160, 40, 10, 1

# Strain parameters

eps_para = -0.006
eps_perp = 0.005

def D_dipole_double_dot(mat_params, lat_params, eps_para, eps_perp, B, d, s, U0, eEz):
    """
    Computes spin splitting and electric dipole matrix matrix elements for the double quantum dot model
    """
    t1 = time.time()
    print('Computing charge and spin splitting, and electric dipole moments for magnetic field B = {0} T and electric field = {1:.1f} mV/nm'.format(B, eEz))
    syst = make_model_LKBP(H_LKBP, mat_params, lat_params, eps_para, eps_perp, B)
    eigs = eigs_double_dot_potential(syst, B, d, s, U0, eEz)
    [Dc, Ds, x01, y01, z01, xl, yl, zl] = D_dip(eigs, lat_params)
    t2 = time.time()
    print('Dc = {0} meV, Ds = {1} meV'.format(Dc, Ds))
    print('x01 = {0:.5f} nm'.format(x01))
    print('y01 = {0:.5f} nm'.format(y01))
    print('z01 = {0:.5f} nm'.format(z01))
    print('xl = {0:.5f} nm'.format(xl))
    print('yl = {0:.5f} nm'.format(yl))
    print('zl = {0:.5f} nm'.format(zl))
    print('Done in {0:3f} s'.format(t2-t1))

D_dipole_double_dot(mat_params=mat_params, lat_params=lat_params, eps_para=eps_para, eps_perp=eps_perp, B=B, d=d, s=s, U0=U0, eEz=eEz)

Material parameters: {'Material': 'Ge', 'ga1': 13.38, 'ga2': 4.24, 'ga3': 5.69, 'ka': 3.41, 'qu': 0.06, 'bv': -2860.0, 'nu': 0.73}
Computing charge and spin splitting, and electric dipole moments for magnetic field B = [0.25, 0.25, 0.05] T and electric field = 1.0 mV/nm
Dc = 2.1098193526956246 meV, Ds = 0.04803857367184239 meV
Dc = 2.1098193526956246 meV, Ds = 0.04803857367184239 meV
x01 = 0.00492 nm
y01 = 0.00926 nm
z01 = 0.00015 nm
xl = 0.00025 nm
yl = 0.00053 nm
zl = 0.00140 nm
Done in 216.193448 s
